<a href="https://colab.research.google.com/github/Chymaster/ML_Learning/blob/main/1L-Conv-Superres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
!unzip DIV2K_train_HR.zip


--2025-01-01 17:27:11--  http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip [following]
--2025-01-01 17:27:12--  https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3530603713 (3.3G) [application/zip]
Saving to: ‘DIV2K_train_HR.zip’

DIV2K_train_HR.zip  100%[===================>]   3.29G  21.0MB/s    in 2m 41s  

2025-01-01 17:29:53 (20.9 MB/s) - ‘DIV2K_train_HR.zip’ saved [3530603713/3530603713]

Archive:  DIV2K_train_HR.zip
   creating: DIV2K_train_HR/
  inflating: DIV2K_train_HR/0103.png  
  inflating: DIV2K_

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
from tqdm import tqdm

class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.conv = nn.Conv2d(3, 3, kernel_size=9, padding=4)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.relu(self.conv(x))

class SRDataset(Dataset):
    def __init__(self, hr_dir, scale_factor=3, target_size=(1280, 720)):
        self.hr_dir = hr_dir
        self.scale_factor = scale_factor
        self.target_size = target_size
        self.image_files = [f for f in os.listdir(hr_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
        self.transform = transforms.ToTensor()

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        hr_img = Image.open(os.path.join(self.hr_dir, self.image_files[idx])).convert('RGB')
        hr_img = hr_img.resize(self.target_size, Image.BICUBIC)
        w, h = hr_img.size
        lr_size = (w // self.scale_factor, h // self.scale_factor)
        lr_img = hr_img.resize(lr_size, Image.BICUBIC)
        lr_img = lr_img.resize((w, h), Image.BICUBIC)
        return self.transform(lr_img), self.transform(hr_img)

def train_srcnn(hr_dir, num_epochs=20, batch_size=128, learning_rate=0.001):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = SRCNN().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    dataset = SRDataset(hr_dir)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Add progress bars for epochs and batches
    epoch_pbar = tqdm(range(num_epochs), desc='Training Progress')
    for epoch in epoch_pbar:
        model.train()
        epoch_loss = 0
        batch_pbar = tqdm(dataloader, leave=False, desc=f'Epoch {epoch+1}')

        for lr_imgs, hr_imgs in batch_pbar:
            lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
            outputs = model(lr_imgs)
            loss = criterion(outputs, hr_imgs)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

            # Update batch progress bar with current loss
            batch_pbar.set_postfix({'loss': f'{loss.item():.4f}'})

        avg_loss = epoch_loss/len(dataloader)
        epoch_pbar.set_postfix({'avg_loss': f'{avg_loss:.4f}'})

    return model

def generate_sr_image(model, lr_image_path, output_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    lr_img = Image.open(lr_image_path).convert('RGB')
    lr_tensor = transforms.ToTensor()(lr_img).unsqueeze(0).to(device)
    with torch.no_grad():
        sr_tensor = model(lr_tensor).clamp(0, 1)
    sr_img = transforms.ToPILImage()(sr_tensor.squeeze(0).cpu())
    sr_img.save(output_path)

if __name__ == '__main__':
    hr_dir = '/content/DIV2K_train_HR'
    model = train_srcnn(hr_dir)
    torch.save(model.state_dict(), 'srcnn_model.pth')
    generate_sr_image(model, 'download.jpeg', 'super_resolved_image.jpg')

Training Progress: 100%|██████████| 20/20 [56:11<00:00, 168.56s/it, avg_loss=0.0049]


FileNotFoundError: [Errno 2] No such file or directory: '/content/download.jpg'

In [4]:
    generate_sr_image(model, 'download.jpeg', 'super_resolved_image.jpg')